# rPPG Predictions

This notebook runs various pretrained models from [rPPG-Toolbox](https://github.com/ubicomplab/rPPG-Toolbox) to extract rPPG signals from videos. The rPPG signals are then saved to disk for further analysis.

In [ ]:
import torch
import respiration.utils as utils

dim = 72
device = utils.get_torch_device()

# Models are trained on different datasets
models = {
    'DeepPhys': [
        'BP4D_PseudoLabel_DeepPhys',
        'MA-UBFC_deepphys',
        'PURE_DeepPhys',
        'SCAMPS_DeepPhys',
        'UBFC-rPPG_DeepPhys',
    ],
}

In [ ]:
from respiration.extractor.deep_phys import DeepPhys


def load_model(name, path) -> torch.nn.Module:
    """
    Load a pretrained model from the rPPG-Toolbox.
    """
    match name:
        case 'DeepPhys':
            loaded_model = DeepPhys(img_size=dim).to(device)
            loaded_model = torch.nn.DataParallel(loaded_model).to(device)
            loaded_model.load_state_dict(torch.load(path, map_location=device))
            return loaded_model
        case _:
            raise ValueError(f'Unknown model: {name}')

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])

In [ ]:
predictions = []

for base_model, models in models.items():
    for model_name in models:
        print(f'Loading model: {base_model} {model_name}')
        model_path = utils.file_path('data', 'rPPG-Toolbox', model_name + '.pth')
        model = load_model(base_model, model_path)

        for (subject, setting) in scenarios:
            print(f'--> Processing: {subject} {setting}')
            frames, meta = dataset.get_video_rgb(subject, setting)

            resized, normalized = utils.preprocess_video_frames(frames, dim)

            # Permute from (T, H, W, C) to (T, C, H, W)
            resized = torch.tensor(resized).permute(0, 3, 1, 2).to(device)
            normalized = torch.tensor(normalized).permute(0, 3, 1, 2).to(device)

            # Concatenate the two inputs
            combined = torch.cat((resized, normalized), dim=1).to(device)

            with torch.no_grad():
                prediction = model(combined)

            prediction = prediction.cpu().numpy().squeeze()

            predictions.append({
                'base_model': base_model,
                'model': model_name,
                'subject': subject,
                'setting': setting,
                'sampling_rate': meta.fps,
                'signal': prediction.tolist(),
            })

In [ ]:
import pandas as pd

df = pd.DataFrame(predictions)
df.head()

In [ ]:
signals_dir = utils.dir_path('outputs', 'signals', mkdir=True)
signals_path = utils.join_paths(signals_dir, 'deep_phys_predictions.csv')
df.to_csv(signals_path, index=False)